# load json

In [1]:
import torch
import numpy as np
import json
import os
import cv2
import random
import glob
import matplotlib.pyplot as plt
import seaborn as sns

import pandas as pd

from  tqdm import tqdm

BLUE=(255, 0, 0)
GREEN = (0, 255, 0)
RED = (0, 0, 255)
YELLOW = (0, 255, 255)
PINK = (255, 0, 255)
BLACK = (0, 0, 0)
ORANGE = (0, 127, 255)
CUSTOM = (255,170,170)
COLOR_CLASS = {0: BLUE, 1:GREEN, 2:RED, 3:YELLOW, 4:PINK, 5:BLACK, 6:ORANGE, 7:CUSTOM}

# def bb_intersection_over_union(boxA, boxB):
#     xA = max(boxA[0], boxB[0])
#     yA = max(boxA[1], boxB[1])
#     xB = min(boxA[2], boxB[2])
#     yB = min(boxA[3], boxB[3])
#     interArea = max(0, xB - xA + 1) * max(0, yB - yA + 1)

#     boxAArea = (boxA[2] - boxA[0] + 1) * (boxA[3] - boxA[1] + 1)
#     boxBArea = (boxB[2] - boxB[0] + 1) * (boxB[3] - boxB[1] + 1)

#     iou = interArea / float(boxAArea + boxBArea - interArea)
#     return iou

def cocoToAbsoluteBox(cocoBox):
    #xywh -> xyxy
    return [cocoBox[0], cocoBox[1], cocoBox[0]+cocoBox[2], cocoBox[1]+cocoBox[3]]


# matplotlib 
def plot(key, list_bbox_):
    fig = plt.figure(figsize=(15,10))

    plt.hist(list_bbox_[key], color = 'blue', edgecolor = 'black',
            bins = int(len(set(list_bbox_[key]))))

    # Add labels
    plt.title('Histogram of {}'.format(key))
    plt.xlabel(key)
    plt.ylabel('count')
#     plt.savefig("d/{}_distribution.png".format(key))


def bb_intersection_over_union(bb1, bb2):
    """
    Calculate the Intersection over Union (IoU) of two bounding boxes.

    Parameters
    ----------
    bb1 : dict
        Keys: {'x1', 'x2', 'y1', 'y2'}
        The (x1, y1) position is at the top left corner,
        the (x2, y2) position is at the bottom right corner
    bb2 : dict
        Keys: {'x1', 'x2', 'y1', 'y2'}
        The (x, y) position is at the top left corner,
        the (x2, y2) position is at the bottom right corner

    Returns
    -------
    float
        in [0, 1]
    """
    bb1 = {'x1':bb1[0], 'x2':bb1[2], 'y1':bb1[1], 'y2':bb1[3]}
    bb2 = {'x1':bb2[0], 'x2':bb2[2], 'y1':bb2[1], 'y2':bb2[3]}
    assert bb1['x1'] < bb1['x2']
    assert bb1['y1'] < bb1['y2']
    assert bb2['x1'] < bb2['x2']
    assert bb2['y1'] < bb2['y2']

    # determine the coordinates of the intersection rectangle
    x_left = max(bb1['x1'], bb2['x1'])
    y_top = max(bb1['y1'], bb2['y1'])
    x_right = min(bb1['x2'], bb2['x2'])
    y_bottom = min(bb1['y2'], bb2['y2'])

    if x_right < x_left or y_bottom < y_top:
        return 0.0

    # The intersection of two axis-aligned bounding boxes is always an
    # axis-aligned bounding box
    intersection_area = (x_right - x_left) * (y_bottom - y_top)

    # compute the area of both AABBs
    bb1_area = (bb1['x2'] - bb1['x1']) * (bb1['y2'] - bb1['y1'])
    bb2_area = (bb2['x2'] - bb2['x1']) * (bb2['y2'] - bb2['y1'])

    # compute the intersection over union by taking the intersection
    # area and dividing it by the sum of prediction + ground-truth
    # areas - the interesection area
    iou = intersection_area / float(bb1_area + bb2_area - intersection_area)
    assert iou >= 0.0
    assert iou <= 1.0
    return iou

# load model

In [2]:
import albumentations as A
from albumentations import *
import torch
import cv2
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import StratifiedKFold
from torchcontrib.optim import SWA
from albumentations.pytorch import ToTensorV2
import math
from torch.nn import BCELoss
import pandas as pd
from sklearn.utils import shuffle
import gc

from layer_3_net.nets import *

model_params = {
    'model_name': 'tf_efficientnet_b4_ns',
    'pretrained': True,
    # 'model_name': 'seresnext50_32x4d',
    #'model_name': 'ViT-B_32',
    #'model_name': 'vit_base_patch32_384',
    'img_size': [256, 256],
    'num_classes': 15,
    'ds': False,
    'ds_blocks': [10, 15],
    'special_augment_prob': 1.,
    'EMA': 1,
    'EMA_model': ''
    
}

training_params = {
    'training_batch_size':120,
    'num_workers': 16,
    'device': torch.device("cuda:0"),
    'device_ids': [0, 1],
    'start_epoch': 1,
    'num_epoch': 50,
    'warm_up': 5,
    'TTA_time': 5
}

def load_checkpoint(path, model):
    ckpt = torch.load(path)
    x = list(model.state_dict().keys())
    y = ckpt['model_state_dict']
    from collections import OrderedDict
    new_state_dict = OrderedDict()
    for i, w in enumerate(y.items()):
        k, v = w
        name = x[i] # remove `module.`
        new_state_dict[name] = v
    model.load_state_dict(new_state_dict)

    return model



def sigmoid(x):
    return 1 / (1 + np.exp(-x))

    
eval_transform = Compose([
        # Resize(model_params['img_size'][0], model_params['img_size'][1], cv2.INTER_AREA),
        # HorizontalFlip(p=0.5),
        # A.GaussNoise(var_limit=(150.0, 200.0), mean=0, p=0.5),
        # A.RandomGamma(gamma_limit=(120, 120), p=0.5),
        # A.RandomBrightnessContrast(contrast_limit=0, brightness_limit=0.2, brightness_by_max=True, p=0.5),
        # A.Rotate(limit=10, p=0.5),
        Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225], max_pixel_value=255.0, p=1.0),
        ToTensorV2(p=1.0),
    ])

In [3]:
path_wildcard = os.path.join('/home/hana/sonnh/kaggle-vin/final_layer3', "*.pt")
ens_dir = glob.glob(path_wildcard)
print(ens_dir)
list_layer3_model = []
for path in ens_dir:
    layer3_model = EfficientNetB3DSPlus(model_params).to(torch.device("cpu"),)
    layer3_model = load_checkpoint( path, layer3_model)
    layer3_model = layer3_model.to(training_params['device'])
    layer3_model = nn.DataParallel(layer3_model, device_ids=[0,1])
    layer3_model.eval()
    list_layer3_model.append(layer3_model)

['/home/hana/sonnh/kaggle-vin/final_layer3/tf_efficientnet_b4_ns_fold-2_epoch-11-1223.pt', '/home/hana/sonnh/kaggle-vin/final_layer3/tf_efficientnet_b4_ns_fold-4_epoch-11-1248.pt', '/home/hana/sonnh/kaggle-vin/final_layer3/tf_efficientnet_b4_ns_fold-5_epoch-8-2985.pt', '/home/hana/sonnh/kaggle-vin/final_layer3/tf_efficientnet_b4_ns_fold-3_epoch-8-1223.pt', '/home/hana/sonnh/kaggle-vin/final_layer3/tf_efficientnet_b4_ns_fold-1_epoch-11-630.pt']


# load json

In [4]:
fold = 1
json_path = '/home/hana/sonnh/kaggle-vin/final/1920_pretrain_train_all/runs_test/fold1.json'
data = json.load(open(json_path, 'r'))
data[0]

{'image_id': '785140281117595d1818b8133d43d3bc.dicom',
 'category_id': 3,
 'bbox': [1823.547, 872.074, 716.453, 448.609],
 'score': 0.03162,
 'conf': [8.696317672729492e-05,
  5.918741226196289e-05,
  6.699562072753906e-05,
  0.0316162109375,
  5.120038986206055e-05,
  0.00011646747589111328,
  4.965066909790039e-05,
  6.568431854248047e-05,
  4.309415817260742e-05,
  0.00017523765563964844,
  0.0003440380096435547,
  0.00011515617370605469,
  4.589557647705078e-05,
  5.936622619628906e-05]}

In [5]:
data_for_layer_3 = {}
for predict_anno in data:
    image_id = predict_anno['image_id']
    if image_id not in data_for_layer_3:
        data_for_layer_3[image_id] = []
    data_for_layer_3[image_id].append(predict_anno)

In [6]:
len(data_for_layer_3)

3000

# predict

In [7]:
def resize(image, size):

    h,w, _ = image.shape


    BLUE = [0.5,0.5,0.5]

    if h > w:
        r = size/h
        new_h = size
        new_w = int(w * r)
        if new_w % 2==1:
            new_w += 1
        image = cv2.resize(image, (new_w, new_h))
        image = cv2.copyMakeBorder( image, 0,0 ,(size - new_w)//2,(size - new_w)//2, cv2.BORDER_CONSTANT, value=BLUE)
    else:
        r = size/w
        new_w = size
        new_h = int(h * r)
        if new_h % 2==1:
            new_h += 1

        image = cv2.resize(image, (new_w, new_h))

        image = cv2.copyMakeBorder( image, (size - new_h)//2,(size - new_h)//2 ,0,0, cv2.BORDER_CONSTANT, value=BLUE)
    return image

In [8]:
for image_id in tqdm(list(data_for_layer_3.keys())):
    black_list = []
    list_small_image = []
    list_bbox = []
    
    #eval
#     img_path_wildcard = os.path.join('dataset/fold2/images/val_all', "{}*".format(image_id))
#     img_paths = glob.glob(img_path_wildcard)
#     image_ori = cv2.imread(img_paths[0])
    
    #test
    image_ori = cv2.imread('/home/hana/sonnh/kaggle-vin/dataset/images_only/test/{}.dicom.png'.format(image_id[:-6]))

    
    
#     print('dataset/fold1/images/val_all/{}.jpg'.format(image_id))
    width, height, _ = image_ori.shape
    out_conf_i = []
    yolo_label_final = []
    for i, predict_anno in enumerate(data_for_layer_3[image_id]):
        out_conf_i.append(predict_anno['conf'])
        xmin, ymin, w, h = predict_anno['bbox']
        
        yolo_label = [0]*14
        yolo_label[predict_anno['category_id']] = 0
        yolo_label_final.append(yolo_label)
        
        if w < 10 or h < 10:
            black_list.append(i)
            xmin, ymin, xmax, ymax = 1,1,2,2
        else:
            xmin = int(xmin)
            ymin = int(ymin)
            xmax = xmin + int(w)
            ymax = ymin + int(h)
            
            
        small_image = image_ori[ymin:ymax, xmin:xmax]
        small_image = resize(small_image, 256)
        small_image = eval_transform(image=small_image)["image"].unsqueeze(0)
        list_small_image.append(small_image)
        
            
        list_bbox.append([xmin/width, ymin/height, xmax/width, ymax/height])
    
    list_small_image  = torch.cat(list_small_image, 0).to(training_params['device'])
    list_bbox = torch.tensor(list_bbox).to(training_params['device'])
    out_conf_i = torch.tensor(out_conf_i).to(training_params['device'])
    yolo_label_final = torch.tensor(yolo_label_final).to(training_params['device'])
    out_conf_final = torch.cat((yolo_label_final, out_conf_i, list_bbox), 1)
    
    layer3_predict_final = None
    
    for layer3_model in list_layer3_model:
        batch_index = 0
        bs = 240
        with torch.no_grad():
            layer3_predict_ = None
            while batch_index + bs < len(list_small_image):

                layer3_predict, _ = layer3_model(list_small_image[batch_index: batch_index + bs], 
                                                    out_conf_final[batch_index: batch_index + bs])
                batch_index += bs
                if layer3_predict_ is None:
                    layer3_predict_ = layer3_predict.detach().cpu()
                else:
                    layer3_predict_ = torch.cat([layer3_predict_, layer3_predict.detach().cpu()], 0)


            if batch_index < len(list_small_image):
                layer3_predict, _ = layer3_model(list_small_image[batch_index:], 
                                                    out_conf_final[batch_index:])

                if layer3_predict_ is None:
                    layer3_predict_ = layer3_predict.detach().cpu()
                else:
                    layer3_predict_ = torch.cat([layer3_predict_, layer3_predict.detach().cpu()], 0)

        layer3_predict = layer3_predict_
        layer3_predict = layer3_predict.squeeze().detach().cpu().numpy()
        layer3_predict = np.atleast_1d(layer3_predict)
        layer3_predict = sigmoid(layer3_predict)
        
#         print(layer3_predict[:5])
        if layer3_predict_final is None :
            layer3_predict_final = layer3_predict
        else:
            layer3_predict_final += layer3_predict
    
    layer3_predict = layer3_predict_final/5
    
    for layer_3_i in range(len(layer3_predict)):
        if np.argmax(layer3_predict[layer_3_i]) == 14  and layer3_predict[layer_3_i][-1] >= 0.95:
            black_list.append(layer_3_i)

    for black_list_i in black_list:
        data_for_layer_3[image_id][black_list_i]['score'] = 0
        

100%|██████████| 3000/3000 [23:13<00:00,  2.15it/s]


# test

In [9]:
import copy

In [10]:
data_json = []
for image_id in data_for_layer_3:
    for anno in data_for_layer_3[image_id]:
        data_json.append(copy.deepcopy(anno))
            
c= 0
for anno in data_json:
    if anno['score'] == 0:
        c += 1
c//3

10736

In [11]:
fold = 1
json_path = '/home/hana/sonnh/kaggle-vin/final/1920_pretrain_train_all/layer3_predict_test/fold1.json'
json.dump(data_json, open(json_path, 'w'))



# eval

In [24]:
import copy

In [25]:
data_json = []
for image_id in data_for_layer_3:
    img_path_wildcard = os.path.join('dataset/fold2/images/val_all', "{}*".format(image_id))
    img_paths = glob.glob(img_path_wildcard)
    
    for anno in data_for_layer_3[image_id]:
        for img_path in img_paths:
            anno['image_id'] = img_path.split('/')[4].split('_')[0]
            
            data_json.append(copy.deepcopy(anno))
            
            
            
        

In [26]:
c= 0
for anno in data_json:
    if anno['score'] == 0:
        c += 1
c//3

10267

In [27]:
fold = 2
epoch = 15
iter_ = '3_3'

json_path = '/home/hana/sonnh/kaggle-vin/final/1920_pretrain_train_all/layer3_predict/fold{}_{}_{}.json'.format(fold, epoch, iter_)
json.dump(data_json, open(json_path, 'w'))